In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import unicodeblock.blocks
import re
from tqdm import tqdm
from random import random

In [2]:
links = {}
expected_unicodes = {}

with open('../language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    
    for pair in pairs:
        expected_unicodes[pair] = pairs[pair]['source']['unicode']
        links[pair] = pairs[pair]['wiki']

links, expected_unicodes

({'Hungarian-German': 'https://en.m.wiktionary.org/wiki/Category:Hungarian_terms_borrowed_from_German',
  'Romanian-Hungarian': 'https://en.m.wiktionary.org/wiki/Category:Romanian_terms_borrowed_from_Hungarian'},
 {'Hungarian-German': 'Latin', 'Romanian-Hungarian': 'Latin'})

In [3]:
# expected Unicode blocks for certain scripts
unicode_block_map = {
    "Latin" : ['BASIC_LATIN', 'LATIN_1_SUPPLEMENT', 'LATIN_EXTENDED_LETTER', 'LATIN_EXTENDED_A', 'LATIN_EXTENDED_B',\
               'LATIN_EXTENDED_C'],
    "Greek" : ['GREEK'],
    "Cyrillic" : ['CYRILLIC', 'CYRILLIC_SUPPLEMENTARY', 'CYRILLIC_EXTENDED_A', 'CYRILLIC_EXTENDED_B'],
    "Arabic" : ['ARABIC', 'ARABIC_SUPPLEMENT', 'ARABIC_PRESENTATION_FORMS_A', 'ARABIC_PRESENTATION_FORMS_B'],
    "Devanagari" : ['DEVANAGARI', 'VEDIC_EXTENSIONS', 'DEVANAGARI_EXTENDED'],
    "Bengali" : ['BENGALI'],
    "Gurmukhi" : ['GURMUKHI'],
    "Tamil" : ['TAMIL'],
    "Telugu" : ['TELUGU'],
    "Malayalam" : ['MALAYALAM'],
    "Myanmar" : ['MYANMAR', 'MYANMAR_EXTENDED_A'],
    "Chinese" : ['CJK_RADICALS_SUPPLEMENT', 'CJK_SYMBOLS_AND_PUNCTUATION', 'CJK_STROKES',\
                 'ENCLOSED_CJK_LETTERS_AND_MONTHS', 'CJK_COMPATIBILITY', 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_A',\
                 'CJK_UNIFIED_IDEOGRAPHS', 'CJK_COMPATIBILITY_IDEOGRAPHS', 'CJK_COMPATIBILITY_FORMS',\
                 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_B', 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_C',\
                 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_D', 'CJK_COMPATIBILITY_IDEOGRAPHS_SUPPLEMENT']
}

In [4]:
def get_all_borrowed_words(dest, source, expected_unicode, invalid=["Unsupported titles/"]):
    title = f"Category:{dest}_terms_borrowed_from_{source}"
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }

    url = f"https://en.wiktionary.org/w/api.php?action=query&list=categorymembers&cmtitle={title}&cmlimit=max"
    
    borrowed_words = []
    while(True):
        r = requests.get(url,params)
        try:
            for cmember in r.json()['query']['categorymembers']:
                if len(cmember['title']) > 1 and \
                    not cmember['title'].startswith('-') and \
                    not cmember['title'].endswith('-') and \
                    not any(inv in cmember['title'] for inv in invalid) and \
                    not any(c.isdigit() for c in cmember['title']) and \
                    not cmember['title'].isupper() and \
                    unicodeblock.blocks.of(cmember['title'][0]) not in ['BASIC_PUNCTUATION']:
                    borrowed_words.append(cmember['title'])
                    if len(borrowed_words) % 1000 == 0:
                        print(f"Got {len(borrowed_words)}")
            cmcontinue = r.json()['continue']['cmcontinue']
            url = url.split("&cmcontinue")[0]
            url+=f"&cmcontinue={cmcontinue}"
        except KeyError:
            break
    print(f"Done, {len(borrowed_words)}")
    return borrowed_words

In [5]:
'''
def get_all_loans_and_homonyms:
    for each word in borrowed words:
        1. get word page from wiktionary
        2. extract source word from source language
        3. extract homonyms from language!=source language

    return loan_pairs, homonyms
'''

def get_source_word(soup, src_lang, expected_unicode, lst_invalid_words):
    for src_soup in soup.find_all("span", class_="etyl"):
        if src_soup != None and src_soup.find(lambda tag: tag.name == 'a' and src_lang.lower() in tag.text.lower()):
            
            src_soup_final = src_soup.find_next("i")
            src_word = src_soup_final.text if src_soup_final != None else ''
            if any(inv in src_word for inv in lst_invalid_words):
                src_soup_final = src_soup.find_next("strong")
                src_word = src_soup_final.text if src_soup_final != None else ''
                source_word = re.split('[/,]',src_word)[0].strip()
            else:
                source_word = re.split('[/,]',src_word)[0].strip()                
                
            if [unicodeblock.blocks.of(c) in unicode_block_map[expected_unicode] for c in source_word].count(True) < len(source_word)*.8:
                source_word = ''
                
            return source_word
    
    return ''

def get_homonym(borrowed_word, soup, src_lang, dest_lang, lst_invalid_words):
    all_homonyms = []
    for header in soup.find_all("h2", id=True):
        if dest_lang.lower() in [header.get('id').lower(), header.text.lower()]:
            try:
                for next_header in header.parent.find_all('h3'):
                    if re.match("Etymology [1-9]",next_header.text):
                        # get next sibling and then find_all span etyl?
                        next_sibs = next_header.find_next_siblings()
                        if len(next_sibs) > 0:
                            all_etyms = next_sibs[0].find_all("span", class_="etyl")
                            if len(all_etyms) > 0:
                                etym = all_etyms[0]
                                for tag in etym.find(lambda tag: tag.name == 'a'):

                                    if src_lang.lower() not in tag.text.lower():
                                        homonym_soup = etym.find_next("i")
                                        ff_word = homonym_soup.text if homonym_soup != None else ''
                                        ff_word = re.split('[,;:]',ff_word)[0].strip()
                                        if any(inv in ff_word for inv in lst_invalid_words):
                                            homonym_soup = etym.find_next("strong")
                                            ff_word = homonym_soup.text if homonym_soup != None else ''
                                            ff_word = re.split('[,;:]',ff_word)[0].strip()

                                        meaning = etym.parent.parent.find_all('ol')[0].text.split('\n')[0]
                                        meaning = re.sub(r'\(.+?\)',r'',meaning)
                                        meaning = re.sub(r'\[.+?\]',r'',meaning)
                                        meaning = meaning.replace('  ', ' ')
                                        meaning = re.split('[.;:]',meaning)[0].strip()

                                        all_homonyms.append([borrowed_word,ff_word,tag.text.lower(),meaning])
            except:
                pass
    
    return all_homonyms

            
def get_all_loans_and_homonyms(borrowed_words, dest_lang, src_lang, expected_unicode, lst_invalid_words=['plural', 'not comparable', 'uncountable', ' spelling', '-person possessive'], min_timeout=5, timeout_after_words=200):
    loan_pairs = [] # [[borrowed_word, source_word]]
    homonyms = [] # 
    p_bar = tqdm(borrowed_words)
    for i, word in enumerate(p_bar):
        while True:
            try:
                params = {
                    'action': 'query',
                    'format': 'json',
                    'prop': 'extracts',
                    'exintro': True,
                    'explaintext': True,
                }

                url = 'https://en.wiktionary.org/w/rest.php/v1/page/' + word + '/html'

                response = requests.get(url, params, timeout=300)

                soup = BeautifulSoup(response.content, 'html.parser')
                source_word = get_source_word(
                    soup, src_lang, expected_unicode, lst_invalid_words)
                homonym = get_homonym(
                    word, soup, src_lang, dest_lang, lst_invalid_words)
                if source_word != '':
                    loan_pairs.append([word, source_word])
                homonyms.extend(homonym)
                p_bar.set_description("Processed word: {}".format(word))
                if i > 0 and i % timeout_after_words == 0:
                    sleep_time = (random() * min_timeout) + min_timeout
                    p_bar.set_description("Collected {} word pairs, sleeping for {} seconds".format(i, sleep_time))
                    time.sleep(sleep_time)
                break
            except Exception as e:
                p_bar.set_description("Word: {}, Error: {}, sleeping for 1 minute".format(word, e))
                time.sleep(60)

    return loan_pairs, homonyms

In [6]:
for pair in links:
    [dest, src] = pair.split('-')
    print(pair)
    words = get_all_borrowed_words(dest, src, expected_unicodes[pair])
    
    overwrite_loans = None
    overwrite_ff = None
    
    if os.path.exists("results/{}.csv".format(pair)):
        overwrite_loans = input("{}.csv exists. Overwrite existing file? (y/n) ".format(pair))
    else:
        overwrite_loans = "y"
        
    if os.path.exists("results/{}_homonyms.csv".format(pair)):
        overwrite_ff = input("{}_homonyms.csv exists. Overwrite existing file? (y/n) ".format(pair))
    else:
        overwrite_ff = "y"
    
    if overwrite_loans == "y" or overwrite_ff == "y":
        print("Getting loan pairs and homonym: {} candidates".format(len(words)))
        loan_words, homonyms = get_all_loans_and_homonyms(words, dest, src, expected_unicodes[pair])

        df_loans = pd.DataFrame(loan_words, columns=['loan_word', 'original_word'])
        df_homonyms = pd.DataFrame(homonyms, columns=['loan_word', 'original_word', 'other_etymology', 'other_meaning'])

        if overwrite_loans == "y":
            df_loans.to_csv("results/{}.csv".format(pair), index=False)
            
        if overwrite_ff == "y":
            df_homonyms.to_csv("results/{}_homonyms.csv".format(pair), index=False)

    print(pair, "done\n")
    print()
        

Hungarian-German
Done, 542
Getting loan pairs and false friends: 542 candidates


Processed word: Category:Hungarian terms borrowed from Old High German: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 542/542 [14:05<00:00,  1.56s/it]


Hungarian-German done


Romanian-Hungarian
Got 1000
Done, 1464
Getting loan pairs and false friends: 1464 candidates


Processed word: Category:Romanian terms partially calqued from Hungarian: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1464/1464 [24:55<00:00,  1.02s/it]

Romanian-Hungarian done


